In [ ]:
t

In [1]:
import sklearn
import sklearn.datasets
import pandas as pd
from sklearn import linear_model
import operator
import numpy as np
import itertools
from sklearn import metrics
from scipy.optimize import minimize 
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression

from fangorn.files_prep import get_data, data_to_pandas
from fangorn.preprocessing import splitting, feature_selection
from fangorn.training import classifiers

from category_encoders import OneHotEncoder
import pymit

import ray
ray.init(num_cpus=20)

2021-02-15 09:19:39,016	WARNING worker.py:1373 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2021-02-15 09:19:39,018	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2021-02-15_09-19-39_017717_1295/logs.
2021-02-15 09:19:39,132	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:35841 to respond...


2021-02-15 09:19:39,262	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:59930 to respond...
2021-02-15 09:19:39,265	INFO services.py:809 -- Starting Redis shard with 10.0 GB max memory.
2021-02-15 09:19:39,289	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2021-02-15_09-19-39_017717_1295/logs.
2021-02-15 09:19:39,305	WARNING services.py:1301 -- Warning: Capping object memory store to 20.0GB. To increase this further, specify `object_store_memory` when calling ray.init() or ray start.
2021-02-15 09:19:39,324	INFO services.py:1475 -- Starting the Plasma object store with 20.0 GB memory using /dev/shm.
2021-02-15 09:19:39,341	WARNING services.py:912 -- Failed to start the reporter. The reporter requires 'pip install psutil'.


{'node_ip_address': '10.96.22.5',
 'redis_address': '10.96.22.5:35841',
 'object_store_address': '/tmp/ray/session_2021-02-15_09-19-39_017717_1295/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-02-15_09-19-39_017717_1295/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2021-02-15_09-19-39_017717_1295'}

In [11]:
# read dataset
all_datasets = get_data.get_all_data(only='ml_challenge')
this_dataset = 'madeline'

# configure dataset
X_all, y_all = data_to_pandas.read_prepare_data(this_dataset)
# X_all = feature_selection.extra_trees_feature_selection(X_all, y_all)
dataset_split_dict = splitting.simple_train_test_val_split(X_all, y_all)

X_train = dataset_split_dict['train']['X']
y_train = dataset_split_dict['train']['y']
X_test = dataset_split_dict['test']['X']
y_test = dataset_split_dict['test']['y']

All ML_CHALLENGE files ready!


In [2]:
X_train = pd.read_csv('X_train_keep_puma38.csv')
X_test = pd.read_csv('X_test_keep_puma38.csv')
y_train = pd.read_csv('y_train_puma38.csv')
y_test = pd.read_csv('y_test_puma38.csv')

In [3]:
X_train

,tau4_disc,tau4_norm,theta2_group_by_tau4_disc_and_mean,theta3_group_by_tau4_disc_and_mean,theta4_group_by_tau4_disc_and_mean,theta5_group_by_tau4_disc_and_mean,theta6_group_by_tau4_disc_and_mean,thetad1_group_by_tau4_disc_and_mean,thetad3_group_by_tau4_disc_and_mean,thetad4_group_by_tau4_disc_and_mean,...,tau4_norm_x_dm4_norm_op_mul_disc,tau4_norm_x_dm5_norm_op_sum_disc,tau4_norm_x_dm5_norm_op_sub_disc,tau4_norm_x_dm5_norm_op_mul_disc,tau4_norm_x_da1_norm_op_sum_disc,tau4_norm_x_da1_norm_op_sub_disc,tau4_norm_x_da1_norm_op_mul_disc,tau4_norm_x_da2_norm_op_sum_disc,tau4_norm_x_da2_norm_op_sub_disc,tau4_norm_x_da2_norm_op_mul_disc
0,7.0,0.624065,-0.088333,-0.059903,-0.044192,-0.036974,-0.022823,-0.027162,0.059600,0.032110,...,6.0,5.0,8.0,2.0,8.0,5.0,6.0,6.0,6.0,4.0
1,4.0,0.365967,0.142450,0.028809,-0.007439,-0.088409,-0.041117,-0.004972,-0.052629,0.115666,...,2.0,4.0,5.0,2.0,2.0,7.0,1.0,4.0,6.0,1.0
2,3.0,0.227629,0.003002,-0.021240,0.020015,-0.082406,-0.024333,0.053877,-0.015534,0.007827,...,2.0,4.0,4.0,2.0,7.0,2.0,3.0,7.0,2.0,3.0
3,9.0,0.802668,0.003497,0.095442,-0.028981,-0.020661,-0.039187,-0.009365,0.008920,0.039989,...,5.0,9.0,5.0,7.0,7.0,7.0,4.0,6.0,8.0,2.0
4,2.0,0.117598,0.087506,-0.071464,0.005565,0.023110,0.027997,-0.034586,-0.041129,0.017857,...,1.0,5.0,2.0,1.0,2.0,5.0,1.0,3.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5729,2.0,0.147271,0.087506,-0.071464,0.005565,0.023110,0.027997,-0.034586,-0.041129,0.017857,...,1.0,4.0,3.0,1.0,4.0,4.0,1.0,3.0,4.0,1.0
5730,7.0,0.612390,-0.088333,-0.059903,-0.044192,-0.036974,-0.022823,-0.027162,0.059600,0.032110,...,6.0,7.0,5.0,5.0,5.0,7.0,2.0,8.0,4.0,6.0
5731,10.0,0.962132,-0.032490,0.005685,-0.110738,0.030240,-0.036637,0.080077,-0.040456,-0.047779,...,5.0,9.0,6.0,8.0,8.0,8.0,6.0,6.0,10.0,2.0
5732,1.0,0.045401,0.006758,0.019967,0.018712,0.018249,0.026089,0.037087,0.034774,-0.009211,...,1.0,5.0,1.0,1.0,5.0,2.0,1.0,3.0,4.0,1.0


In [4]:
def numpy_discretize(X_train, X_test, max_gran=10):
    """
    multi-granularity discretization
    method. The basic idea is simple: instead of using a fine-tuned
    granularity, we discretize each numerical feature into several, rather
    than only one, categorical features, each with a different granularity.
    
    min granularity = 3
    
    Sometimes de edge values did not permit to execute correct discretization
    if this happens the step is not executed
    """
    
    # separa dados numericos que precisam de binarizacao
    is_numeric = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    numeric_features = X_train.select_dtypes(include=is_numeric)
    discrete_features = []
    print(f"Discretizing {len(numeric_features.columns)} features...")
    feat_count = 0
    for feat in numeric_features:
        if feat_count % 50 == 0:
            print(f" Working in {feat}")
        X_train_np = X_train[[feat]].to_numpy()
        X_test_np = X_test[[feat]].to_numpy()
        for gran in range(3, max_gran+1):
            try:
                D_train = np.zeros([X_train.shape[0], 1])
                D_test = np.zeros([X_test.shape[0], 1])
                # calc numpy histogram and apply to features
                hist, bin_edges = np.histogram(X_train_np[:, 0], bins=gran)
                D_train[:, 0] = np.digitize(X_train_np[:,0], bin_edges, right=False)
                D_test[:, 0] = np.digitize(X_test_np[:,0], bin_edges, right=False)

                # apply back to pandas
                X_train[f"{feat}_{gran}"] = D_train
                X_test[f"{feat}_{gran}"] = D_test
            except:
                print(f"Not possible to correct work on cut {feat} > {gran}")
                break
        
        feat_count += 1
        X_train = X_train.drop(feat, axis=1)
        X_test = X_test.drop(feat, axis=1)
        
    return X_train, X_test

In [5]:
X_train_discrete, X_test_discrete = numpy_discretize(X_train.copy(), X_test.copy())

Discretizing 1840 features...
 Working in tau4_disc
 Working in theta1_x_tau4_disc_op_mul
 Working in thetad1_x_tau4_op_sub
 Working in thetad6_x_tau4_norm_op_sum
 Working in tau4_x_da3_op_mul
 Working in tau4_x_theta6_disc_op_sum
 Working in tau4_x_dm1_disc_op_mul
 Working in tau4_x_db4_disc_op_sum
 Working in tau4_x_tau3_norm_op_sum
 Working in tau4_x_db4_norm_op_mul
 Working in da2_x_tau4_disc_op_sub
 Working in db4_x_tau4_disc_op_mul
 Working in thetad1_disc_x_tau4_norm_op_mul
 Working in tau4_disc_x_dm2_disc_op_sub
 Working in tau4_disc_x_db4_disc_op_div
 Working in tau4_disc_x_tau4_norm_op_sum
 Working in tau4_disc_x_db5_norm_op_mul
 Working in thetad3_norm_x_tau4_norm_op_mul
 Working in tau4_norm_x_da5_norm_op_sub
 Working in dm1_group_by_tau4_disc_and_mean_disc
 Working in theta4_x_thetad2_disc_op_mul_disc
 Working in thetad3_x_db3_norm_op_sub_disc
 Working in tau3_x_da5_norm_op_div_disc
 Working in tau4_x_da2_op_sub_disc
 Working in tau4_x_theta4_disc_op_div_disc
 Working in t

In [6]:
def hjmi_selector(X, y, bins, max_features):
    
    X = X.to_numpy()
    Y = y.to_numpy().ravel()

    [tmp, features] = X.shape
    D = np.zeros([tmp, features])

    for i in range(features):
        N, E = np.histogram(X[:,i], bins=bins)
        D[:,i] = np.digitize(X[:,i], E, right=False)

    selected_features = []
    j_h = 0
    hjmi = None
    for i in range(0,max_features):
        JMI = np.zeros([features], dtype=np.float)
        for X_k in range(features):
            if X_k in selected_features:
                continue
            jmi_1 = pymit.I(D[:,X_k], Y, bins=[bins,2])
            jmi_2 = 0
            for X_j in selected_features:
                tmp1 = pymit.I(D[:,X_k], D[:,X_j], bins=[bins,bins])
                tmp2 = pymit.I_cond(D[:,X_k], D[:,X_j], Y, bins=[bins,bins,2])
                jmi_2 += tmp1 - tmp2
            if len(selected_features) == 0:
                JMI[X_k] += j_h + jmi_1
            else:
                JMI[X_k] += j_h + jmi_1 - jmi_2/len(selected_features)
        
        f = JMI.argmax()
        j_h = JMI[f]
        if (hjmi == None) or ((j_h - hjmi)/hjmi > 0.03):
            r = 0
            if hjmi != None:
                r = ((j_h - hjmi)/hjmi) 

            hjmi = j_h
            selected_features.append(f)
            print("{:0>3d} {:>3d} {} - {}".format(len(selected_features), f, j_h, r))
        else:
            return selected_features

In [7]:
%%time
selected_features = hjmi_selector(X_train_discrete.copy(), y_train.copy(), bins=10, max_features=300)

001 6694 0.23847811506799615 - 0
002 711 0.587068861881326 - 1.4617305521470505
003 5751 0.9713531464587024 - 0.654581275773843
004 703 1.382220482011072 - 0.42298451088595695
005 4679 1.7958136807036544 - 0.29922375198117596
006 671 2.146617655671121 - 0.19534541847905448
007 4703 2.501451221848415 - 0.16529891349764314
008 12063 2.8828216396954116 - 0.1524596660194668
009 5743 3.2442830634715873 - 0.12538459500892551
010 4671 3.614564039213088 - 0.11413337507772107
011 4711 3.976245443297294 - 0.1000622482159553
012 679 4.339236986383113 - 0.09129002428602812
013 8087 4.704862611563707 - 0.08426034953332988
014 12055 5.069126048039829 - 0.07742275737889308
015   7 5.423563449704838 - 0.06992081047226402
016 8095 5.787909730000381 - 0.06717839362889193
017 4678 6.147882120069588 - 0.062193850087773124
018 4670 6.502508701711076 - 0.057682723044382885
019 12087 6.84556485527229 - 0.05275750780171077
020 13135 7.198785193004371 - 0.051598421050680576
021 12062 7.551678126713163 - 0.0490

In [8]:
selected_features

[6694,
 711,
 5751,
 703,
 4679,
 671,
 4703,
 12063,
 5743,
 4671,
 4711,
 679,
 8087,
 12055,
 7,
 8095,
 4678,
 4670,
 12087,
 13135,
 12062,
 6695,
 12095,
 13127,
 6703,
 702,
 14079,
 8063,
 710,
 8055,
 8086,
 15,
 14087]

In [13]:
selected_features = [1902, 687, 1479, 1535, 1303, 1903, 686, 423, 567, 1534, 684, 1901, 1471, 685, 1533, 1087, 1487, 1900, 1863, 1470, 1531, 1486, 1478, 671, 1523, 1302, 1431, 1583, 2022]

In [9]:
filtered_train = X_train_discrete[X_train_discrete.columns[selected_features]]
filtered_test = X_test_discrete[X_test_discrete.columns[selected_features]]

In [10]:
from fangorn.training import classifiers

this_clf = classifiers.random_forest_classifier(train_set= [filtered_train, y_train],
                     test_set= [filtered_test, y_test],
                     features= filtered_test.columns,
                     target= 'thetadd3_bin',
                     test_metrics= ['auc'],
                     project_name= 'explore_kit'
                     ) 
this_auc = this_clf['calc_metrics']['auc']

In [11]:
this_auc

0.943079505165833

In [10]:
filtered_train

,theta2_x_theta3_op_sum_10,theta2_disc_x_theta3_disc_op_sub_9,theta3_disc_10,theta2_disc_x_theta3_norm_op_sum_10,theta2_x_theta3_op_sum_disc_10,theta2_norm_x_theta3_norm_op_sum_9,theta2_norm_x_theta3_norm_op_sum_10,theta2_x_theta3_disc_op_sum_10,theta2_norm_x_theta3_norm_op_sum_disc_10,theta2_x_theta3_op_sum_9,...,theta2_disc_x_theta3_disc_op_sum_10,theta2_norm_x_theta3_norm_op_sum_disc_9,theta2_x_theta3_disc_op_sub_10,theta2_disc_x_theta3_disc_op_sum_disc_10,theta2_disc_x_theta3_norm_op_sub_10,theta2_x_theta3_op_sum_8,theta3_disc_x_theta2_norm_op_mul_disc_10,theta3_disc_x_theta2_norm_op_sum_10,theta2_disc_x_theta3_disc_op_sum_8,theta2_norm_x_theta3_norm_op_sum_8
0,6.0,8.0,4.0,8.0,6.0,6.0,6.0,5.0,6.0,6.0,...,6.0,5.0,8.0,6.0,9.0,5.0,4.0,4.0,5.0,5.0
1,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,8.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,4.0,7.0,2.0,4.0,4.0,3.0,4.0,3.0,4.0,3.0,...,3.0,3.0,8.0,3.0,5.0,3.0,1.0,2.0,3.0,3.0
3,5.0,1.0,9.0,1.0,5.0,4.0,5.0,6.0,5.0,4.0,...,5.0,4.0,1.0,5.0,1.0,4.0,1.0,8.0,4.0,4.0
4,5.0,7.0,4.0,6.0,5.0,5.0,5.0,4.0,5.0,5.0,...,5.0,4.0,7.0,5.0,7.0,4.0,3.0,4.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5729,7.0,5.0,8.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,...,7.0,6.0,4.0,7.0,7.0,6.0,5.0,7.0,6.0,6.0
5730,9.0,6.0,8.0,9.0,9.0,8.0,9.0,8.0,9.0,8.0,...,8.0,8.0,5.0,8.0,8.0,7.0,7.0,8.0,7.0,7.0
5731,4.0,8.0,1.0,6.0,4.0,4.0,4.0,2.0,4.0,4.0,...,4.0,3.0,10.0,4.0,7.0,4.0,1.0,1.0,3.0,4.0
5732,3.0,5.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,2.0,7.0,3.0,3.0,2.0,1.0,3.0,2.0,2.0


In [21]:
def get_dummies(X_train, X_test):
    ohe = OneHotEncoder(cols=X_train.columns).fit(X=X_train)
    X_train = ohe.transform(X_train)
    X_test = ohe.transform(X_test)
    return X_train, X_test

final_X_train, final_X_test = get_dummies(filtered_train.copy(), filtered_test.copy())


In [49]:
X = final_X_train.to_numpy()
[tmp, features] = X.shape
D = np.zeros([tmp, features])
bins = 2
for i in range(features):
    N, E = np.histogram(X[:,i], bins=2)
    D[:,i] = np.digitize(X[:,i], E, right=False)


In [51]:
hjmi_selector(final_X_train.copy(), y_train.copy(), bins=2, max_features=300)

[[3. 1. 1. ... 1. 1. 1.]
 [1. 3. 1. ... 1. 1. 1.]
 [3. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 3. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 3. ... 1. 1. 1.]]
001 119 0.02300667735340972 - 0
002 108 0.046401166807680166 - 1.0168565019147908
003 203 0.07582262760604848 - 0.6340672621512304
004 206 0.09993868406021007 - 0.31805883303677307
005 177 0.12309585809961139 - 0.23171381789908113
006 134 0.14465429754337655 - 0.1751353764179432
007 264 0.16288740276715322 - 0.12604606661139273
008 147 0.18035896831276102 - 0.10726161292278277
009 100 0.1967068210705373 - 0.0906406424405102
010 136 0.21474743753544126 - 0.0917132225853762
011 283 0.2311906047157449 - 0.07656979458760671
012 224 0.2473158362451812 - 0.06974864549215873
013  13 0.26306459219651585 - 0.06367872025680486
014 205 0.27934979934878085 - 0.061905735835781135
015  37 0.2946753055981063 - 0.05486134690288734
016 216 0.30947440758969924 - 0.05022172442157997
017   5 0.3241142314048669 - 0.04730544257015627
018 110 0.3389348946

[119,
 108,
 203,
 206,
 177,
 134,
 264,
 147,
 100,
 136,
 283,
 224,
 13,
 205,
 37,
 216,
 5,
 110,
 4,
 101,
 69,
 176,
 11,
 128]

In [52]:
pp = [119,
 108,
 203,
 206,
 177,
 134,
 264,
 147,
 100,
 136,
 283,
 224,
 13,
 205,
 37,
 216,
 5,
 110,
 4,
 101,
 69,
 176,
 11,
 128]

In [58]:
dummy_final_train = final_X_train[final_X_train.columns[pp]]
dummy_final_test = final_X_test[final_X_test.columns[pp]]

In [63]:
filtered_train = X_train_discrete.copy()
filtered_test = X_test_discrete.copy()

In [68]:
final_X_train['x'] = _treino[_treino.columns[23736]]
final_X_test['x'] = final_X_test['191_9_4'] | final_X_test['191_6_4']

In [11]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(dummy_final_train, y_train)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, neigh.predict(dummy_final_test))

NameError: name 'dummy_final_train' is not defined

In [61]:
def run_initial_logit(X_train, y_train, X_test, y_test):
    clf = LogisticRegression(max_iter=3000).fit(X_train, y_train[y_train.columns[0]])
    all_preds = clf.predict_proba(X_test)[:, 1]
    logloss = metrics.log_loss(y_test[y_test.columns[0]], all_preds)
    fpr, tpr, thresholds = metrics.roc_curve(y_test[y_test.columns[0]], all_preds, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    return clf, logloss, auc

start_classifier, logloss, auc = run_initial_logit(final_X_train.copy(), y_train.copy(), final_X_test.copy(), y_test.copy())
print(f"LOGLOSS: {logloss} - AUC: {auc}")

LOGLOSS: 0.659345746147491 - AUC: 0.6935060975609757


In [66]:
def get_dummies(X_train, X_test):
    ohe = OneHotEncoder(cols=X_train.columns).fit(X=X_train)
    X_train = ohe.transform(X_train)
    X_test = ohe.transform(X_test)
    return X_train, X_test

final_X_train, final_X_test = get_dummies(filtered_train.copy(), filtered_test.copy())


In [33]:
def _combine_all_features(current_training_set, pairwise_cols):
    all_features_combined = []
    for feature_pair in pairwise_cols:
        feature_name = str(feature_pair)
        if feature_name not in current_training_set.columns:
            combined_dict = {}
            combined_features = current_training_set[feature_pair[0]] | current_training_set[feature_pair[1]]
            combined_dict['feature_name'] = str(feature_pair)
            combined_dict['feature_value'] = combined_features
            if combined_features.equals(current_training_set[feature_pair[0]]) or combined_features.equals(current_training_set[feature_pair[1]]):
                continue
            else:
                all_features_combined.append(combined_dict)
    return all_features_combined

In [54]:
all_columns = list(final_X_train)
pairwise_cols = list(itertools.combinations(all_columns, 2))

# print("Combinando features do nivel...")
# all_features_combined = _combine_all_features(final_X_train, pairwise_cols)

In [37]:
_treino = final_X_train.copy()

In [42]:
hjmi_selector(_treino.copy(), y_train.copy(), bins=2, max_features=300)

001 23736 0.049014304416390486 - 0


KeyboardInterrupt: 

2953    0
1560    0
2098    0
71      0
1636    0
       ..
2680    0
1854    0
217     0
2164    1
1697    0
Name: ('191_9_4', '191_6_4'), Length: 2009, dtype: int64

In [38]:
for feat in all_features_combined:
    _treino[feat['feature_name']] = feat['feature_value']

2020-09-13 13:45:12,493	ERROR import_thread.py:89 -- ImportThread: Connection closed by server.


KeyboardInterrupt: 

2020-09-13 13:45:12,493	ERROR worker.py:1716 -- listen_error_messages_raylet: Connection closed by server.
2020-09-13 13:45:12,496	ERROR worker.py:1616 -- print_logs: Connection closed by server.


---

In [31]:
def run_initial_logit(X_train, y_train, X_test, y_test):
    clf = LogisticRegression().fit(X_train, y_train[y_train.columns[0]])
    all_preds = clf.predict_proba(X_test)[:, 1]
    logloss = metrics.log_loss(y_test[y_test.columns[0]], all_preds)
    fpr, tpr, thresholds = metrics.roc_curve(y_test[y_test.columns[0]], all_preds, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    return clf, logloss, auc

start_classifier, logloss, auc = run_initial_logit(final_X_train.copy(), y_train.copy(), final_X_test.copy(), y_test.copy())
print(f"LOGLOSS: {logloss} - AUC: {auc}")

LOGLOSS: 0.6593040135656834 - AUC: 0.6934654471544716


/home/fernando.favoretti/anaconda3/envs/basic/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [62]:

def to_iterator(obj_ids):
    """
    # https://github.com/ray-project/ray/issues/5554
    """
    while obj_ids:
        done, obj_ids = ray.wait(obj_ids)
        yield ray.get(done[0])

    
def score_one_pair_parallel(combined_features, bsum, y_train):

    def _calc_logloss(y_true, preds):
        return metrics.log_loss(y_true[y_true.columns[0]], preds['preds'])

    def _obj(x, 
             y_train,
             combined_features,
             bsum):

        # add x to bsum
        this_w = np.array(x).reshape(-1,1)
        this_value = np.array(combined_features).reshape(1,-1)
        bsum_with_new_feature = bsum + np.matmul(this_w.T, this_value)
        this_preds = 1/(1 + np.exp(-bsum_with_new_feature)) 
        preds = pd.DataFrame(this_preds.reshape(-1,1), columns=['preds'])
        return _calc_logloss(y_train, preds)
    
    
    result = minimize(_obj, 1, args=(y_train,
                             combined_features['feature_value'],
                             bsum))

    this_coef = result['x'][0]
    this_logloss = result['fun']
    dict_result_combination = {"coef":this_coef,
                               "logloss" : this_logloss,
                               "feature": combined_features['feature_name']}

    return dict_result_combination


def iter_one_level(X_train, y_train, X_test, y_test, coef_dict, intercept):
    
    def _combine_all_features(current_training_set, pairwise_cols):
        all_features_combined = []
        for feature_pair in pairwise_cols:
            feature_name = str(feature_pair)
            if feature_name not in current_training_set.columns:
                combined_dict = {}
                combined_features = current_training_set[feature_pair[0]] | current_training_set[feature_pair[1]]
                combined_dict['feature_name'] = str(feature_pair)
                combined_dict['feature_value'] = combined_features
                if combined_features.equals(current_training_set[feature_pair[0]]) or combined_features.equals(current_training_set[feature_pair[1]]):
                    continue
                else:
                    all_features_combined.append(combined_dict)
        return all_features_combined
    
    def _chunker(seq, size):
        return (seq[pos:pos + size] for pos in range(0, len(seq), size))
    
    all_columns = list(X_train)
    pairwise_cols = list(itertools.combinations(all_columns, 2))
    all_results = {}
    col_coefs = np.array(list(coef_dict.values())).reshape(1,-1)
    bsum = np.add(np.matmul(col_coefs, X_train.values.T), intercept)
    
    print("Combinando features do nivel...")
    all_features_combined = _combine_all_features(X_train, pairwise_cols)
    print(f"{len(all_features_combined)} pares criados")
    
    all_results = []
    
    print("Iniciando paralelismo do nivel...")
    chunk = 0
    chuncksize = int(10e3)
    for group in _chunker(all_features_combined, chuncksize):
        print(f"\t working in chunk {chunk}")
        score_one_pair_parallel_ray = ray.remote(score_one_pair_parallel)
        results = [score_one_pair_parallel_ray.remote(ray.put(one_pair), bsum, ray.put(y_train)) for one_pair in group]
        for x in tqdm(to_iterator(results), total=len(results)):
            pass

        all_results.append(ray.get(results))
        chunk += chuncksize
        
    return sum(all_results, [])

def predict_logit(coef_dict, intercept, X_test, y_test):
    col_coefs = np.array(list(coef_dict.values())).reshape(1,-1)
    bsum = np.add(np.matmul(col_coefs, X_test.values.T), intercept)
    this_preds = 1/(1 + np.exp(-bsum)) 
    preds = pd.DataFrame(this_preds.reshape(-1,1), columns=['preds'])
    logloss = metrics.log_loss(y_test[y_test.columns[0]], preds['preds'])
    fpr, tpr, thresholds = metrics.roc_curve(y_test[y_test.columns[0]],  preds['preds'], pos_label=1)
    auc = metrics.auc(fpr, tpr)
    return logloss, auc

def beam_search(X_train, y_train, X_test, y_test):
    
    def _choose_best_feature(level_results):
        min_logloss = 9999
        bsf_feature = None
        bst_coef = None
        for dict_feature in level_results:
            if  dict_feature['logloss'] < min_logloss:
                min_logloss = dict_feature['logloss']
                bsf_feature = dict_feature['feature']
                bst_coef = dict_feature['coef']
        print(f"Level - choose {bsf_feature} -{bst_coef} -{min_logloss}")
        return bsf_feature, bst_coef, min_logloss
    
    current_training_set = X_train.copy()
    current_test_set = X_test.copy()
    start_classifier, start_logloss, start_auc = run_initial_logit(current_training_set, y_train, current_test_set, y_test)
    
    coef_dict = dict(list(zip(current_training_set.columns,start_classifier.coef_[0])))
    intercept = start_classifier.intercept_[0]
    
    print(f"Start logloss : {start_logloss} - Start AUC {start_auc}")
    last_logloss = start_logloss
    this_logloss = -np.inf
    accepted_features = []
    while this_logloss <= last_logloss:
        # eval one level
        level_results = iter_one_level(current_training_set, y_train, current_test_set, y_test, coef_dict, intercept)
        bst_feature, this_coef, _ = _choose_best_feature(level_results)
        
        # update X_train an X_test
        current_training_set[str(bst_feature)] = current_training_set[str(eval(bst_feature)[0])] | current_training_set[str(eval(bst_feature)[1])]
        current_test_set[str(bst_feature)] = current_test_set[str(eval(bst_feature)[0])] | current_test_set[str(eval(bst_feature)[1])]
        
        # retrain logit with new feature
#         this_clf, this_logloss, this_auc = run_initial_logit(current_training_set, y_train, current_test_set, y_test)
#         coef_dict = dict(list(zip(current_training_set.columns,this_clf.coef_[0])))
#         intercept = start_classifier.intercept_[0]
        coef_dict[bst_feature] = this_coef
        
        this_logloss, this_auc = predict_logit(coef_dict, intercept, current_test_set, y_test)
        print(f" new  logloss: {this_logloss} - new auc {this_auc}")
        
        coef_dict[bst_feature] = this_logloss
    
        current_logloss_diff = last_logloss - this_logloss
        
        if this_logloss > last_logloss:
            current_training_set =  current_training_set.drop([str(bst_feature)], axis=1)
            current_test_set =  current_test_set.drop([str(bst_feature)], axis=1)
            coef_dict.pop(str(bst_feature), None)
            
        else:
            last_logloss = this_logloss
            
        print(f"logloss gain with {bst_feature}: {current_logloss_diff}")
    return current_training_set, current_test_set, coef_dict, intercept   

In [64]:
complete_X_train, complete_X_test, coef_dict, intercept = beam_search(dummy_final_train, y_train, dummy_final_test, y_test)

Start logloss : 0.6549597108668569 - Start AUC 0.677545731707317
Combinando features do nivel...
271 pares criados
Iniciando paralelismo do nivel...
	 working in chunk 0


/home/fernando.favoretti/anaconda3/envs/basic/lib/python3.7/site-packages/ray/pyarrow_files/pyarrow/serialization.py:176: FutureWarning: The SparseSeries class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  if isinstance(obj, pd.SparseSeries):
/home/fernando.favoretti/anaconda3/envs/basic/lib/python3.7/site-packages/ray/pyarrow_files/pyarrow/serialization.py:165: FutureWarning: The SparseDataFrame class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  if isinstance(obj, pd.SparseDataFrame):


ArrowIOError: Broken pipe